In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from pydrake.geometry import StartMeshcat

from iiwa_batter import PACKAGE_ROOT, CONTROL_DT, PITCH_DT, CONTACT_DT, NUM_JOINTS
from iiwa_batter.robot_constraints.get_joint_constraints import JOINT_CONSTRAINTS
from iiwa_batter.physics import find_ball_initial_velocity, PITCH_START_POSITION
from iiwa_batter.save_load import load_control_vector_trajectory, load_dataviewer_trajectory

from iiwa_batter.swing_optimization.full_trajectory import run_full_trajectory, setup_simulator, make_torque_trajectory

In [ ]:
meshcat = StartMeshcat()

In [ ]:
# Load control vector
trajectory_settings = {
    "robot": "iiwa14",
    "pitch_speed_mph": 90,
    "target_position": [0, 0, 0.6],
}
optimization_name = "naive_full_trajectory_step_309"
#optimization_name = "crash_trajectory"

robot = trajectory_settings["robot"]
pitch_speed_mph = trajectory_settings["pitch_speed_mph"]
target_position_y = trajectory_settings["target_position"][1]
target_position_z = trajectory_settings["target_position"][2]

best_control_vector, _, _, _ = load_control_vector_trajectory(trajectory_settings, optimization_name)

simulator, station = setup_simulator(dt=CONTACT_DT, meshcat=meshcat)
ball_initial_velocity, time_of_flight = find_ball_initial_velocity(pitch_speed_mph, 
                                                         trajectory_settings["target_position"])
trajectory_timesteps = np.arange(0, time_of_flight, CONTROL_DT)
robot_constraints = JOINT_CONSTRAINTS
torque_trajectory = make_torque_trajectory(best_control_vector, NUM_JOINTS, trajectory_timesteps)
torque_trajectory[0] = np.array([1e8] * NUM_JOINTS)

contact_loss = run_full_trajectory(meshcat, simulator, station, best_control_vector[:NUM_JOINTS], [PITCH_START_POSITION, ball_initial_velocity], time_of_flight, robot_constraints, torque_trajectory)

simulator, station = setup_simulator(dt=PITCH_DT, meshcat=meshcat)
pitch_loss = run_full_trajectory(meshcat, simulator, station, best_control_vector[:NUM_JOINTS], [PITCH_START_POSITION, ball_initial_velocity], time_of_flight, robot_constraints, torque_trajectory)

print(f"Contact dt loss: {contact_loss}")
print(f"Pitch dt loss: {pitch_loss}")
print(f"Difference: {contact_loss - pitch_loss}")

In [ ]:
setup_simulator(dt=CONTACT_DT, meshcat=None)